In [ ]:
%pip install brian2
%pip install brian2tools
%pip install scipy
!pip install --upgrade scipy

     |████████████████████████████████| 1.2MB 4.3MB/s 
  Created wheel for brian2: filename=Brian2-2.4.2-cp37-cp37m-linux_x86_64.whl size=1257119 sha256=ceef65f4618312a4ba5fac17455192721530d5a4205d249ee0490a5b425f2ef9
  Stored in directory: /root/.cache/pip/wheels/5d/4b/cf/ef50e71df5479d67f981cbb17b1d842381019d27ba144d57de
Successfully built brian2
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 27.4MB 75.1MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd
from scipy import io
from scipy import sparse
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/MyDrive/Figures_Package')
import seaborn as sns

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
initial_v = np.load('/content/gdrive/MyDrive/Figures_Package/Networks/initial_voltage.npy')

In [ ]:
w_EE = sparse.load_npz('/content/gdrive/MyDrive/Figures_Package/Session_Matrices/w_EE1.npz')
w_IE = sparse.load_npz('/content/gdrive/MyDrive/Figures_Package/Session_Matrices/w_IE_sparse.npz')
w_EI = sparse.load_npz('/content/gdrive/MyDrive/Figures_Package/Session_Matrices/w_EI_sparse.npz')
w_II = sparse.load_npz('/content/gdrive/MyDrive/Figures_Package/Session_Matrices/w_II_sparse.npz')

In [ ]:
j_EE, i_EE, w_EE = sparse.find(w_EE)
j_IE, i_IE, w_IE = sparse.find(w_IE)
j_EI, i_EI, w_EI = sparse.find(w_EI)
j_II, i_II, w_II = sparse.find(w_II)

In [ ]:
N_E = 32000
f = 0.2
num_targetted = int(N_E*f)
#all_ex = np.arange(N_E)
#aux = np.zeros((N_E, N_E))
#aux[j_EE, i_EE] = w_EE
#target_ids = np.random.choice(N_E, num_targetted)

In [ ]:
#we need the firings distribution for the target neurons

def process_params(av, av2):
    var = (av2 - av**2) 
    mu = np.log(av**2/(av**2 + var)**(1/2))
    sigma = np.log(1 + var/av**2) 
    return mu, sigma

avs = np.array([0.37, 0.66, 0.44, 0.54])
avs2 = np.array([0.26, 0.65, 0.49, 0.53])

lognormal_mu = []
lognormal_std = []

for i in range(len(avs)):
    mu, sigma = process_params(avs[i], avs2[i])
    lognormal_mu.append(mu)
    lognormal_std.append(sigma)
    
ee_mu = lognormal_mu[0]
ee_std = lognormal_std[0]
ie_mu = lognormal_mu[1]
ie_std = lognormal_std[1]
ei_mu = lognormal_mu[2]
ei_std = lognormal_std[2]
ii_mu = lognormal_mu[3]
ii_std = lognormal_std[3]

In [ ]:
num_targetted

640

In [ ]:
#define network parameters

#total size of the network
N = 40000

#threshold, reset and refractory parameters
v_theta = 33*mV
v_reset = 24.75*mV
tau_ref = 1*ms
v_0 = -1000*mV

#indices that delimit inhibitory and excitatory populations 20/80
first_inh = 0
last_inh = int(0.2*N)
first_exc = last_inh
last_exc = N


#these are additional parameters to play with resizing of weights (in paper would be 1)
g_E = 1
g_I = 1


#External drive to each population

H_in = 57.8*mV
H_ex = 77.6*mV

tau_in = np.log((H_in - v_reset)/(H_in - v_theta))*second;
tau_ex = np.log((H_ex - v_reset)/(H_ex - v_theta))*second;


#clear brian scope to reset any past variables
start_scope()

#define the model that each neuron will follow
tau = 10*ms
eqs = '''
dv/dt = -(v - H_ext)/tau : volt (unless refractory)
H_ext : volt
'''

#the original network is defined
all_neurons = NeuronGroup(N, eqs, threshold = 'v>v_theta', reset = 'v=v_reset', refractory = tau_ref, method = 'exact')

#to work with subpopulations bria-n uses slicing notation
in_neurons = all_neurons[first_inh:last_inh]
ex_neurons = all_neurons[first_exc:last_exc]
target_neurons = ex_neurons[:num_targetted]

all_neurons.v = initial_v*mV

#define external drive according to supp table 1
in_neurons.H_ext = H_in
ex_neurons.H_ext = H_ex


#initialize the synaptic object for the different subpopulation pairs
S_EE = Synapses(ex_neurons, ex_neurons, 'w : volt', on_pre ='v += w')
S_IE = Synapses(ex_neurons, in_neurons, 'w : volt', on_pre ='v += w')
S_EI = Synapses(in_neurons, ex_neurons,'w : volt', on_pre ='v -= w')
S_II = Synapses(in_neurons, in_neurons, 'w : volt', on_pre ='v -= w')


#initialize the synaptic object for the different subpopulation pairs
#connect them according to probabilities in supp table 1
S_EE.connect(i=i_EE, j=j_EE)
S_IE.connect(i=i_IE, j=j_IE)
S_EI.connect(i=i_EI, j=j_EI)
S_II.connect(i=i_II, j=j_II)

S_EE.w = w_EE*mV
S_IE.w = w_IE*volt
S_EI.w = w_EI*volt
S_II.w = w_II*volt


#create additional synapses between a fraction f of the excitatory and the rest of the exc neurons
S_target = Synapses(ex_neurons, target_neurons, 'w : volt', on_pre ='v += w')
S_target.connect(p = 0.04)
S_target.w = 'exp(ee_mu + ee_std*randn())*mV'


#monitors spiking events
M_in = SpikeMonitor(in_neurons)
M_ex = SpikeMonitor(ex_neurons)
M_ex_targ = SpikeMonitor(ex_neurons[:num_targetted])
M_ex_not_targ = SpikeMonitor(ex_neurons[num_targetted:])

In [ ]:
S_target.w.shape

(8189297,)

In [ ]:
#N_E = 32000
#N_I = 8000
#super_path = '/content/gdrive/MyDrive/Figures_Package/Session_Matrices/'

#w_IE = zeros((N_I, N_E))
#w_IE[S_IE.j, S_IE.i] = S_IE.w
#w_IE = sparse.csc_matrix(w_IE)
#path_IE = super_path + 'w_IE_sparse.npz'
#sparse.save_npz(path_IE, w_IE)

#w_EI = zeros((N_E, N_I))
#w_EI[S_EI.j, S_EI.i] = S_EI.w
#w_EI = sparse.csc_matrix(w_EI)
#path_EI = super_path + 'w_EI_sparse.npz'
#sparse.save_npz(path_EI, w_EI)
#w_II = zeros((N_I, N_I))

#w_II[S_II.j, S_II.i] = S_II.w
#w_II = sparse.csc_matrix(w_II)
#sparse.save_npz(path_II, w_II)
#path_II = super_path + 'w_II_sparse.npz'

In [ ]:
defaultclock.dt = 0.005*ms
total_time = 10000*ms

run(total_time, report = 'text')

Starting simulation at t=0. s for a duration of 10. s
164.575 ms (1%) simulated in 10s, estimated 9m 58s remaining.
0.32765 s (3%) simulated in 20s, estimated 9m 50s remaining.
0.490185 s (4%) simulated in 30s, estimated 9m 42s remaining.
0.652315 s (6%) simulated in 40s, estimated 9m 33s remaining.
0.81424 s (8%) simulated in 50s, estimated 9m 24s remaining.
0.976865 s (9%) simulated in 1m 0s, estimated 9m 14s remaining.
1.14044 s (11%) simulated in 1m 10s, estimated 9m 4s remaining.
1.302915 s (13%) simulated in 1m 20s, estimated 8m 54s remaining.
1.465725 s (14%) simulated in 1m 30s, estimated 8m 44s remaining.
1.62896 s (16%) simulated in 1m 40s, estimated 8m 34s remaining.
1.79086 s (17%) simulated in 1m 50s, estimated 8m 24s remaining.
1.953465 s (19%) simulated in 2m 0s, estimated 8m 14s remaining.
2.115645 s (21%) simulated in 2m 10s, estimated 8m 4s remaining.
2.27852 s (22%) simulated in 2m 20s, estimated 7m 54s remaining.
2.44213 s (24%) simulated in 2m 30s, estimated 7m 44s

In [ ]:
def process_firings(M_in, M_ex, M_ex_targ, M_ex_not_targ, time):
    global fraction
    count_in = np.array(M_in.count)/(time/second)
    count_ex = np.array(M_ex.count)/(time/second)
    count_ex_targ = np.array(M_ex_targ.count)/(time/second)
    count_ex_not_targ = np.array(M_ex_not_targ.count)/(time/second)
    count_in_path = '/content/gdrive/MyDrive/Figures_Package/Ex_Target04/FR_in_f' + fraction + '.npy'
    count_ex_path = '/content/gdrive/MyDrive/Figures_Package/Ex_Target04/FR_ex_f' + fraction + '.npy'
    count_ex_targ_path = '/content/gdrive/MyDrive/Figures_Package/Ex_Target04/FR_ex_targ_f' + fraction + '.npy'
    count_ex_not_targ_path = '/content/gdrive/MyDrive/Figures_Package/Ex_Target04/FR_ex_not_targ_f' + fraction + '.npy'
    np.save(count_in_path, count_in)
    np.save(count_ex_path, count_ex)
    np.save(count_ex_targ_path, count_ex_targ)
    np.save(count_ex_not_targ_path, count_ex_not_targ)

In [ ]:
time=total_time
fraction = '02'
process_firings(M_in, M_ex, M_ex_targ, M_ex_not_targ, time)

In [ ]:
FR_1 = np.load('/content/gdrive/MyDrive/Figures_Package/Session_Matrices/FR_ex_session1.npy')
FR_2 = np.load('/content/gdrive/MyDrive/Figures_Package/Ex_Target04/FR_ex_f02.npy')

In [ ]:
norm1 = np.linalg.norm(FR_1)
norm2 = np.linalg.norm(FR_2)
dot = np.dot(FR_1, FR_2)/(norm1*norm2)

In [ ]:
dot

0.6302877749923685